# Emotion Chat Bot

In [1]:
from operator import itemgetter

import torch
from libs import (
    EmotionModel,
    ResponseGeneratorPipeline,
    SimilarityAnalyser,
    generate_dummy_representation,
    get_sentiment_composition,
)
from libs.FullModel import (
    create_candidates_buffer,
    get_top_emotion,
)
from sympy.core.random import randint
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)
from unsloth import FastLanguageModel

ImportError: cannot import name 'candidates' from 'transformers.utils.import_utils' (/home/hermeschen/Repo/chat-bot/.venv/lib/python3.12/site-packages/transformers/utils/import_utils.py)

## Load Each Module

### Response Generator

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
    attn_implementation="flash_attention_2",
    pretraining_tp=1,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

In [ ]:
response_generator = ResponseGeneratorPipeline(
    model,
    tokenizer,
    framework="pt",
    task="conversation-generation",
    num_workers=16,
    torch_dtype="auto",
    add_special_tokens=True,
    truncation=False,
    padding=True,
)

In [ ]:
FastLanguageModel.for_inference(response_generator.model)
response_generator.model = torch.compile(
    response_generator.model, mode="reduce-overhead"
)

### Sentiment Analyzer

In [ ]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [ ]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [ ]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=sentiment_analysis_model,
    tokenizer=sentiment_analysis_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
sentiment_analyzer.model = torch.compile(
    sentiment_analyzer.model, mode="reduce-overhead"
)

In [ ]:
print(sentiment_analyzer.model)

### Emotion Predictor

In [ ]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [ ]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [ ]:
emotion_predictor = pipeline(
    "sentiment-analysis",
    model=emotion_predictor_model,
    tokenizer=emotion_predictor_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [ ]:
emotion_predictor.model = torch.compile(emotion_predictor.model, mode="reduce-overhead")

In [ ]:
print(emotion_predictor.model)

### Emotion Model

In [ ]:
emotion_model = EmotionModel.from_pretrained(
    "hermeschen1116/emotion_model_for_emotion_chat_bot"
)

In [ ]:
emotion_model = torch.compile(emotion_model, mode="reduce-overhead")

In [ ]:
print(emotion_model)

### Similarity Analyzer

In [ ]:
threshold: float = 0.5

In [ ]:
similarity_analyzer = SimilarityAnalyser(threshold)

## Combine All Modules

### Initialize

In [ ]:
chat_buffer_size: int = 10

In [ ]:
streamer = TextStreamer(
    tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [ ]:
ideal_bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
ideal_bot_emotion_representation

In [ ]:
bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
bot_emotion_representation

In [ ]:
system_prompt: str = input("Enter your system prompt: ").strip()
bot_message: str = "Talk to me..."

In [ ]:
chat_buffer: list = [
    {"role": "system", "content": {"emotion": "", "dialog": system_prompt}}
]

In [ ]:
chat_buffer

In [ ]:
while True:
    user_response: str = input(f"Bot: {bot_message}").strip()
    if user_response == "quit":
        break

    user_emotion: list = sentiment_analyzer(user_response)
    chat_buffer.append(
        {
            "role": "user",
            "content": {
                "emotion": get_top_emotion(user_emotion),
                "dialog": user_response,
            },
        }
    )

    user_emotion_composition: Tensor = get_sentiment_composition(user_emotion)

    bot_emotion_representation = emotion_model.forward(
        user_emotion_composition, bot_emotion_representation
    )

    bot_chat_simulations: list = create_candidates_buffer(chat_buffer)
    bot_chat_simulations = [
        chat[0] for chat in response_generator(bot_chat_simulations, streamer=streamer)
    ]
    print()
    # bot_chat_simulations = [chat[0] for chat in response_generator(bot_chat_simulations)]
    bot_chat_simulations = list(
        filter(lambda chat: chat[-1]["content"]["dialog"] != "", bot_chat_simulations)
    )

    bot_response_simulations: list = [
        chat[-1]["content"] for chat in bot_chat_simulations
    ]

    user_future_emotion_composition_simulations: dict = {
        response["emotion"]: get_sentiment_composition(
            emotion_predictor(response["dialog"])
        )
        for response in bot_response_simulations
    }

    bot_future_emotion_representations: dict = {
        k: emotion_model.forward(v, bot_emotion_representation)
        for k, v in user_future_emotion_composition_simulations.items()
    }

    emotion_representation_similarity_scores: list = similarity_analyzer(
        list(bot_future_emotion_representations.values()),
        ideal_bot_emotion_representation,
    ).tolist()

    emotions: list = [
        "neutral",
        "anger",
        "disgust",
        "fear",
        "happiness",
        "sadness",
        "surprise",
    ]
    bot_best_response_emotion: str = emotions[
        max(enumerate(emotion_representation_similarity_scores), key=itemgetter(1))[0]
    ]

    if len(chat_buffer) == chat_buffer_size + 1:
        chat_buffer.pop(1)
    chat_buffer.append(
        {"role": "bot", "content": {"emotion": bot_best_response_emotion, "dialog": ""}}
    )

    chat_buffer = response_generator(chat_buffer, streamer=streamer)[0]
    print()
    if chat_buffer[-1]["content"]["dialog"] == "":
        chat_buffer[-1]["content"]["dialog"] = dict(
            filter(lambda x: x.key() == bot_best_response_emotion, bot_chat_simulations)
        ).value()
    bot_message = chat_buffer[-1]["content"]["dialog"]